# Simulation (Olivetti dataset)

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_olivetti_faces

In [2]:
from hals import NMF, PMF

In [3]:
pd.options.display.float_format = '{:.3f}'.format

Parameters
----

In [4]:
max_iter = 100

In [5]:
l1term  = (0.0, 0.0)

Input data
----

In [6]:
fpath = '../input/CLUTO_tr41.pkl'
f = open(fpath, 'rb')
dataset = pickle.load(f)
f.close()

X = dataset['X'].astype('float').toarray()
y = dataset['y']
row, col = X.shape
dim = len(np.unique(y))
print('The shape of X: {} * {}'.format(row, col))
print('class: {}'.format(dim))

The shape of X: 7454 * 878
class: 10


Simulation
----

In [7]:
original_ret = ['NA']  * 4 
normalized_ret = ['NA'] * 4
positive_ret = ['NA'] * 4
proposed_ret = ['NA'] * 4

In [8]:
elem_W = row * dim
elem_H = col * dim

In [9]:
original = NMF(n_components=dim, max_iter=max_iter, l1term=l1term, calc_obj=True, calc_pgrad=True, stopkkt_flag=False, eps=0.0, normal_flag=False).fit(X)

# Relative objective function value
original_obj = original.get_obj()[-1]

# Relative squared projected gradient norm
original_pgrad = original.get_pgrad_norm()[-1]

# Ratio of zero elements in W	
original_W = original.get_basis()
original_zeroratio_W = (elem_W - len(np.nonzero(original_W)[0])) / elem_W

# Ratio of zero elements in H
original_H = original.get_coef()
original_zeroratio_H = (elem_H - len(np.nonzero(original_H)[0])) / elem_H

# Result
original_ret = [original_obj, original_pgrad, original_zeroratio_W, original_zeroratio_H]

In [10]:
normalized = NMF(n_components=dim, max_iter=max_iter, l1term=l1term, calc_obj=True, calc_pgrad=True, stopkkt_flag=False, eps=0.0, normal_flag=True).fit(X)

# Relative objective function value
normalized_obj = normalized.get_obj()[-1]

# Relative squared projected gradient norm
normalized_pgrad = normalized.get_pgrad_norm()[-1]

# Ratio of zero elements in W	
normalized_W = normalized.get_basis()
normalized_zeroratio_W = (elem_W - len(np.nonzero(normalized_W)[0])) / elem_W

# Ratio of zero elements in H
normalized_H = normalized.get_coef()
normalized_zeroratio_H = (elem_H - len(np.nonzero(normalized_H)[0])) / elem_H

# Result
normalized_ret = [normalized_obj, normalized_pgrad, normalized_zeroratio_W, normalized_zeroratio_H]

In [11]:
positive = PMF(n_components=dim, max_iter=max_iter, l1term=l1term, calc_obj=True, calc_pgrad=True, stopkkt_flag=False, normal_flag=False).fit(X)

# Relative objective function value
positive_obj = positive.get_obj()[-1]

# Relative squared projected gradient norm
positive_pgrad = positive.get_pgrad_norm()[-1]

# Ratio of zero elements in W	
positive_W = positive.get_basis()
positive_zeroratio_W = (elem_W - len(np.nonzero(positive_W)[0])) / elem_W

# Ratio of zero elements in H
positive_H = positive.get_coef()
positive_zeroratio_H = (elem_H - len(np.nonzero(positive_H)[0])) / elem_H

# Result
positive_ret = [positive_obj, positive_pgrad, positive_zeroratio_W, positive_zeroratio_H]

In [12]:
proposed = NMF(n_components=dim, max_iter=max_iter, l1term=l1term, calc_obj=True, calc_pgrad=True, stopkkt_flag=False, normal_flag=True).fit(X)

# Relative objective function value
proposed_obj = proposed.get_obj()[-1]

# Relative squared projected gradient norm
proposed_pgrad = proposed.get_pgrad_norm()[-1]

# Ratio of zero elements in W	
proposed_W = proposed.get_basis()
proposed_zeroratio_W = (elem_W - len(np.nonzero(proposed_W)[0])) / elem_W

# Ratio of zero elements in H
proposed_H = proposed.get_coef()
proposed_zeroratio_H = (elem_H - len(np.nonzero(proposed_H)[0])) / elem_H

# Result
proposed_ret = [proposed_obj, proposed_pgrad, proposed_zeroratio_W, proposed_zeroratio_H]

Result
--------

In [13]:
index = [
    'Relative objective function value', 
    'Relative squared projected gradient norm',
    'Ratio of zero elements in W',
    'Ratio of zero elements in H']
columns = [
    'Original',
    'Normalized',
    'Positive',
    'Proposed'
]

df = pd.DataFrame(index=index, columns=columns)
df['Original'] = original_ret
df['Normalized'] = normalized_ret
df['Positive'] = positive_ret
df['Proposed'] = proposed_ret

df

,Original,Normalized,Positive,Proposed
Relative objective function value,0.703,0.703,0.703,0.703
Relative squared projected gradient norm,0.000,0.000,0.002,0.000
Ratio of zero elements in W,0.489,0.489,0.000,0.489
Ratio of zero elements in H,0.489,0.489,0.000,0.489
